# Writing custom Jaxpr interpreters in JAX

JAX offers several composable function transformations (`jit`, `grad`, `vmap`,
etc.) that enable writing concise, accelerated code. 

Here we show how to add your own function transformations to the system, by writing a custom Jaxpr interpreter. And we'll get composability with all the other transformations for free.

In [1]:
import numpy as onp
import jax
import jax.numpy as np
from jax import jit, grad, vmap
from jax import random

## What is JAX doing?

JAX provides a NumPy-like API for numerical computing which can be used as is, but JAX's true power comes from composable function transformations. Take the `jit` function transformation, which takes in a function and returns a semantically identical function but is lazily compiled by XLA for accelerators.


In [2]:
x = random.normal(random.PRNGKey(0), (5000, 5000))
def f(w, b, x):
  return np.tanh(np.dot(x, w) + b)
fast_f = jit(f)

/usr/local/google/home/sharadmv/workspace/jax/jax/lib/xla_bridge.py:115: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


When we call `fast_f`, what happens? JAX traces the function and constructs an XLA computation graph. The graph is then JIT-compiled and executed. Other transformations work similarly in that they first trace the function and handle the output trace in some way. To learn more about Jax's tracing machinery, you can refer to the ["How it works"](https://github.com/google/jax#how-it-works) section in the README.

## Jaxpr tracer

A tracer of special importance in Jax is the Jaxpr tracer, which records ops into a Jaxpr (Jax expression). A Jaxpr is a data structure that can be evaluated like a mini functional programming language and 
thus Jaxprs are a useful intermediate representation
for function transformation. 



To get a first look at Jaxprs, consider the `make_jaxpr` transformation. `make_jaxpr` is essentially a "pretty-printing" transformation:
it transforms a function into one that, given example arguments, produces a Jaxpr representation of its computation.
Although we can't generally use the Jaxprs that it returns, it is useful for debugging and introspection.
Let's use it to look at how some example Jaxprs
are structured.

In [3]:
def examine_jaxpr(jaxpr):
  print("invars:", jaxpr.invars)
  print("outvars:", jaxpr.outvars)
  print("constvars:", jaxpr.constvars)
  print("freevars:", jaxpr.freevars)
  for eqn in jaxpr.eqns:
    print("equation:", eqn.invars, eqn.primitive, eqn.outvars, eqn.params)
  print()
  print("jaxpr:", jaxpr)

def foo(x):
  return x + 1
print("foo")
print("=====")
examine_jaxpr(jax.make_jaxpr(foo)(5))

print()

def bar(w, b, x):
  return np.dot(w, x) + b + np.ones(5), x
print("bar")
print("=====")
examine_jaxpr(jax.make_jaxpr(bar)(np.ones((5, 10)), np.ones(5), np.ones(10)))

foo
=====
invars: [a]
outvars: [b]
constvars: []
freevars: []
equation: [a, 1] add [b] {}

jaxpr: { lambda  ;  ; a.
  let b = add a 1
  in [b] }


bar
=====
invars: [a, b, c]
outvars: [g, c]
constvars: [f]
freevars: []
equation: [a, c] dot_general [d] {'dimension_numbers': (((1,), (0,)), ((), ())), 'precision': None}
equation: [d, b] add [e] {}
equation: [e, f] add [g] {}

jaxpr: { lambda f ;  ; a b c.
  let d = dot_general[ dimension_numbers=(((1,), (0,)), ((), ()))
                       precision=None ] a c
      e = add d b
      g = add e f
  in [g, c] }



* `jaxpr.invars` - the `invars` of a Jaxpr are a list of the input variables to Jaxpr, analogous to arguments in Python functions
* `jaxpr.outvars` - the `outvars` of a Jaxpr are the variables that are returned by the Jaxpr. Every Jaxpr has multiple outputs.
* `jaxpr.constvars` - the `constvars` are a list of variables that are also inputs to the Jaxpr, but correspond to constants from the trace (we'll go over these in more detail later)
* `jaxpr.freevars` - these can arise when nesting `jit` and `pmap` transformations; we won't worry about them in this colab.
* `jaxpr.eqns` - a list of equations, which are essentially let-bindings. Each equation is list of input variables, a list of output variables, and a *primitive*, which is used to evaluate inputs to produce outputs. Each equation also has a set of `params`, a dictionary of keyword arguments to the primitive.

All together, a Jaxpr encapsulates a simple program that can be evaluated with inputs to produce an output. We'll go over how exactly to do this later. The important thing to note now is that a Jaxpr is a data structure that can be manipulated and evaluated in whatever way we want.

### Why are Jaxprs useful?

Jaxprs are simple program representations that are easy to transform. And because Jax lets us stage out Jaxprs from Python functions, it gives us a way to transform numerical programs written in Python.

## Your first interpreter: `invert`

Let's try to implement a simple function "inverter", which takes in the output of the original function and returns the inputs that produced those outputs. For now, let's focus on simple, unary functions which are composed of other invertible unary functions.

Goal:
```python
def f(x):
  return np.exp(np.tanh(x))
f_inv = inverse(f)
assert np.allclose(f_inv(f(1.0)), 1.0)
```

The way we'll implement this is by (1) tracing `f` into a Jaxpr, then (2) interpreting the Jaxpr *backwards*. While interpreting the Jaxpr backwards, for each equation we'll look up the primitive's inverse in a table and apply it.

### 1. Tracing a function

We can't use `make_jaxpr` for this, because we need to pull out constants created during the trace to pass into the Jaxpr. However, we can write a function that does something very similar to `make_jaxpr`.

In [4]:
# Importing Jax functions useful for tracing/interpreting.
import numpy as onp
from functools import wraps

from jax import api_util
from jax import core
from jax import lax
from jax import linear_util as lu
from jax import tree_util
from jax.abstract_arrays import ShapedArray
from jax.interpreters import partial_eval as pe
from jax.util import safe_map

In [5]:
def make_jaxpr2(fun):
  
  def pv_like(x):
    # ShapedArrays are abstract values that carry around
    # shape and dtype information
    aval = ShapedArray(onp.shape(x), onp.result_type(x))
    return pe.PartialVal((aval, core.unit))

  @wraps(fun)
  def jaxpr_const_maker(*args, **kwargs):
    # Set up fun for transformation
    wrapped = lu.wrap_init(fun)
    # Flatten input args
    jax_args, in_tree = tree_util.tree_flatten((args, kwargs))
    # Transform fun to accept flat args
    # and return a flat list result
    jaxtree_fun, out_tree = api_util.flatten_fun(wrapped, in_tree) 
    # Abstract and partial-val's flat args
    pvals = safe_map(pv_like, jax_args)
    # Trace function into Jaxpr
    jaxpr, _, consts = pe.trace_to_jaxpr(jaxtree_fun, pvals) 
    return jaxpr, consts, (in_tree, out_tree())
  return jaxpr_const_maker

This function first flattens its arguments into a list, which are the abstracted and wrapped as partial values. The `pe.trace_to_jaxpr` function is used to then trace a function into a Jaxpr
from a list of partial value inputs.

In [6]:
def f(x):
  return np.exp(np.tanh(x))
jaxpr, consts, _ = make_jaxpr2(f)(np.ones(5))
print(jaxpr)
print(consts)

{ lambda  ;  ; a.
  let b = tanh a
      c = exp b
  in [c] }

()


This particular function doesn't have any example constants, but in general, this is how you both trace into a Jaxpr and extract the constants.

### 2. Evaluating a Jaxpr


Before we write a custom Jaxpr interpreter, let's first implement the "default" interpreter, `eval_jaxpr`, which evaluates the Jaxpr as-is, computing the same values that the original, un-transformed Python function would. 

To do this, we first create an environment to store the values for each of the variables, and update the environment with each equation we evaluate in the Jaxpr.

In [7]:
def eval_jaxpr(jaxpr, consts, *args):
  # Mapping from variable -> value
  env = {}
  
  def read(var):
    # Literals are values baked into the Jaxpr
    if type(var) is core.Literal:
      return var.val
    return env[var]

  def write(var, val):
    env[var] = val

  # Bind args and consts to environment
  write(core.unitvar, core.unit)
  safe_map(write, jaxpr.invars, args)
  safe_map(write, jaxpr.constvars, consts)

  # Loop through equations and evaluate primitives using `bind`
  for eqn in jaxpr.eqns:
    # Read inputs to equation from environment
    invals = safe_map(read, eqn.invars)  
    # `bind` is how a primitive is called
    outvals = eqn.primitive.bind(*invals, **eqn.params)
    # Primitives may return multiple outputs or not
    if not eqn.primitive.multiple_results: 
      outvals = [outvals]
    # Write the results of the primitive into the environment
    safe_map(write, eqn.outvars, outvals) 
  # Read the final result of the Jaxpr from the environment
  return safe_map(read, jaxpr.outvars) 

In [8]:
jaxpr, consts, _ = make_jaxpr2(f)(np.ones(5))
eval_jaxpr(jaxpr, consts, np.ones(5))

[DeviceArray([2.14168763, 2.14168763, 2.14168763, 2.14168763, 2.14168763],
             dtype=float32)]

Notice that `eval_jaxpr` will always return a list even if the original function does not. To "unflatten" the list into what the function was originally supposed to return, we can use the `out_tree` object returned by `trace`. 

Furthermore, this interpreter does not handle `subjaxprs`, which we will not cover in this guide. You can refer to `core.eval_jaxpr` ([link](https://github.com/google/jax/blob/master/jax/core.py#L185-L212)) to see the edge cases that this interpreter does not cover.


### Custom `inverse` Jaxpr interpreter

An `inverse` interpreter doesn't look too different from `eval_jaxpr`. We'll first set up the registry which will map primitives to their inverses. We'll then write a custom interpreter that looks up primitives in the registry.

It turns out that this interpreter will also look similar to the "transpose" interpreter used in reverse-mode autodifferentiation [found here](https://github.com/google/jax/blob/master/jax/interpreters/ad.py#L141-L187).


In [9]:
inverse_registry = {}

We'll now register inverses for some of the primitives. By convention, primitives in Jax end in `_p` and a lot of the popular ones live in `lax`.

In [10]:
inverse_registry[lax.exp_p] = np.log
inverse_registry[lax.tanh_p] = np.arctanh

`inverse` will first trace the function, then custom-interpret the Jaxpr. Let's set up a simple skeleton.

In [11]:
def inverse(fun):
  @wraps(fun)
  def wrapped(*args, **kwargs):
    # Since we assume unary functions, we won't
    # worry about flattening and
    # unflattening arguments
    jaxpr, consts, _ = make_jaxpr2(fun)(*args, **kwargs)
    out = inverse_jaxpr(jaxpr, consts, *args)
    return out[0]
  return wrapped

Now we just need to define `inverse_jaxpr`, which will walk through the Jaxpr backward and invert primitives when it can.

In [12]:
def inverse_jaxpr(jaxpr, consts, *args):
  env = {}
  
  def read(var):
    if type(var) is core.Literal:
      return var.val
    return env[var]

  def write(var, val):
    env[var] = val
  # Args now correspond to Jaxpr outvars
  write(core.unitvar, core.unit)
  safe_map(write, jaxpr.outvars, args)
  safe_map(write, jaxpr.constvars, consts)

  # Looping backward
  for eqn in jaxpr.eqns[::-1]:
    #  outvars are now invars 
    invals = safe_map(read, eqn.outvars)
    if eqn.primitive not in inverse_registry:
      raise NotImplementedError("{} does not have registered inverse.".format(
          eqn.primitive
      ))
    # Assuming a unary function 
    outval = inverse_registry[eqn.primitive](*invals)
    safe_map(write, eqn.invars, [outval])
  return safe_map(read, jaxpr.invars)

That's it!

In [13]:
def f(x):
  return np.exp(np.tanh(x))
f_inv = inverse(f)
assert np.allclose(f_inv(f(1.0)), 1.0)

/usr/local/google/home/sharadmv/workspace/jax/jax/numpy/lax_numpy.py:220: RuntimeWarning: divide by zero encountered in arctanh
  return _dtype(op(*args))


Importantly, you can trace through a Jaxpr interpreter.

In [14]:
jax.make_jaxpr(inverse(f))(f(1.))

{ lambda  ;  ; a.
  let b = log a
      c = abs b
      d = le c 1.0
      e = add b 1.0
      f = sub 1.0 b
      g = div e f
      h = log g
      i = mul h 0.5
      j = tie_in b nan
      k = broadcast[ sizes=() ] j
      l = select d i k
  in [l] }

That's all it takes to add a new transformation to a system, and you get composition with all the others for free! For example, we can use `jit`, `vmap`, and `grad` with `inverse`!

In [15]:
jit(vmap(grad(inverse(f))))((np.arange(5) + 1.) / 5.)

DeviceArray([ 0.       , 15.58493137,  2.25512528,  1.31550276,
              1.       ], dtype=float32)

## Exercises for the reader

* Handle primitives with multiple arguments where inputs are partially known, for example `lax.add_p`, `lax.mul_p`.
* Handle `xla_call` and `xla_pmap` primitives, which will not work with both `eval_jaxpr` and `inverse_jaxpr` as written.